In [1]:
from torchvision import datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
from torch import nn
from torchvision import transforms
import torch.nn.functional as F
from torch import optim

epochs=20
batch_size=4
learning_rate=0.0001

#prep dataset
trans = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
training_set=datasets.FashionMNIST('data',train=True,download=True,transform=trans)
testing_set=datasets.FashionMNIST('data',train=False,download=True,transform=trans)
train_data=DataLoader(training_set,shuffle=True,batch_size=batch_size)
test_data=DataLoader(testing_set,shuffle=False,batch_size=batch_size)


In [2]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv_1 = nn.Conv2d(1,6,5)
    self.conv_2 = nn.Conv2d(6,16,5)
    self.fc_1 = nn.Linear(256,128)
    self.fc_2 = nn.Linear(128,52)
    self.fc_3 = nn.Linear(52,10)

  def forward(self,x):
    x = F.relu(self.conv_1(x))
    x = F.max_pool2d(x,2)
    x = F.relu(self.conv_2(x))
    x = F.max_pool2d(x,2)
    x=x.view(x.size(0), -1)
    x = F.relu(self.fc_1(x))
    x = F.relu(self.fc_2(x))
    x= self.fc_3(x)
    return x

In [3]:
my_net = CNN()
print(my_net)
cal_loss=nn.CrossEntropyLoss()
optimizer=optim.Adam(my_net.parameters(),lr=learning_rate)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
my_net.to(device)

CNN(
  (conv_1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv_2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc_1): Linear(in_features=256, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=52, bias=True)
  (fc_3): Linear(in_features=52, out_features=10, bias=True)
)


CNN(
  (conv_1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv_2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc_1): Linear(in_features=256, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=52, bias=True)
  (fc_3): Linear(in_features=52, out_features=10, bias=True)
)

In [ ]:
#training
for epoch in range(epochs):
  print ("epoch........."+str(epoch))
  running_loss=0
  training_accuracy=0
  my_net.train()
  for img,lbl in train_data:
    optimizer.zero_grad()
    score=my_net(img)
    loss=cal_loss(score,lbl)
    loss.backward()
    optimizer.step()
    running_loss += loss.detach().numpy()
    training_accuracy += (score.argmax(dim=1) == lbl).sum().numpy()

  test_running_loss=0
  test_accuracy=0
  my_net.eval()
  with torch.no_grad():
    for img,lbl in test_data:
        score = my_net(img)
        loss = cal_loss(score, lbl)
        test_running_loss += loss.detach().numpy()
        test_accuracy += (score.argmax(dim=1) == lbl).sum().numpy()

  train_loss = running_loss/len(train_data)
  test_loss = test_running_loss/len(test_data)
  training_accuracy /= len(training_set)
  test_accuracy /= len(testing_set)
  print('    loss     (train, test): {:.4f},  {:.4f}'.format(train_loss, test_loss))
  print('    accuracy (train, test): {:.4f},  {:.4f}'.format(training_accuracy, test_accuracy)) 


epoch.........0
